In [ ]:
import json
from datetime import datetime
from pyquery import PyQuery as pq
from math import log, tan, pi, sqrt, pow, isnan

In [ ]:
def load(filename):
    with open(filename) as f:
        return json.load(f)

In [ ]:
def type_cat(c,m):
    c=color(c)
    m=topmark(m)
    cat=None
    if 'black' in c and 'red' in c:
        t='buoy_isolated_danger'
    if 'white' in c and 'red' in c and (m is None or 'sphere' in m):
        t='buoy_safe_water'
    elif c=='yellow':
        t='buoy_special_purpose'
    elif 'black' in c and 'yellow' in c:
        t='buoy_cardinal'
        if c=='black;yellow':
          cat='north'
        elif c=='black;yellow;black':
          cat='east'
        elif c=='yellow;black':
          cat='south'
        elif c=='yellow;black;yellow':
          cat='west'
    elif 'green' in c or 'red' in c:
        t='buoy_lateral'
        if c.startswith('green'):
            if 'red' in c:
                cat='preferred_channel_port'
            else:
                cat='starboard'
        elif c.startswith('red'):
            if 'green' in c:
                cat='preferred_channel_starboard'
            else:
                cat='port'
    return t,cat

In [ ]:
colors={1:'white', 2:'black', 3:'red', 4:'green', 6:'yellow'}
def color(s,osm=0):
    if s=='#': return
    for k,v in colors.items():
        s=s.replace(str(k),v)
    return s.replace(',',';')

In [ ]:
shapes={1:'conical', 2:'can', 3:'spherical', 4:'pillar', 5:'spar', 6:'barrel', 7:'super-buoy'}
def shape(s):
    if s=='#': return
    return shapes[int(s)]

In [ ]:
patterns={1:'horizontal', 2:'vertical'}
def pattern(s):
    if s=='#': return
    return patterns[int(s)]

In [ ]:
topmarks={1:'cone, point up', 2:'cone, point down', 3:'sphere', 5:'cylinder', 7:'x-shape',
          10:'2 cones point together', 11:'2 cones base together', 13:'2 cones up', 14:'2 cones down',
          98:'cylinder over sphere', 99:'cone, point up over sphere'}
def topmark(s):
    if s=='#': return
    return topmarks[int(s)]

In [ ]:
light_characters={1:'F',2:'Fl',3:'LFl',4:'Q',5:'VQ',6:'UQ',
                      7:'Iso',8:'Oc',9:'IQ',10:'IVQ',11:'IUQ',
                      12:'Mo',13:'FFl',14:'FLFl',15:'OcFl',
                      16:'FLFl',17:'OcAlt',18:'LFlAlt',19:'FlAlt',
                      25:'Q+LFl',26:'VQ+LFl',27:'UQ+LFl',28:'Al',29:'F+FlAlt'}
def light_chr(s):
    if s=='#': return
    return light_characters[int(s)]

In [ ]:
def light_per(s):
    if s=='#': return
    return str(int(s))

In [ ]:
def light_grp(s):
    if s=='#': return
    return s.replace('(1)','').replace('(','').replace(')','') or None

In [ ]:
def latlon_to_grid(lat, lon):
    f = 20037508.34
    x = (lon * f) / 180
    y = log(tan((90 + lat) * pi / 360)) / (pi / 180)
    y = (y * f) / 180
    return x, y

In [ ]:
def distance(a,b):
    a,b=latlon_to_grid(*a),latlon_to_grid(*b)
    return sqrt(pow(a[0]-b[0],2)+pow(a[1]-b[1],2))

In [ ]:
def load_data(filename):
    data=load(filename)
    now=datetime.now().date().isoformat()
    source=f'https://data.overheid.nl/dataset/2c5f6817-d902-4123-9b1d-103a0a484979 {now}'
    
    points=[]
    for f in data['features']:
        try:
            p=f['properties']
            ll=p['y_wgs84'],p['x_wgs84']
            tags={'ll':ll}
            n=p['benaming']
            assert n and '#' not in n
            tags['seamark:name']=n
            tags['seamark:source']=source
            tags['seamark:source:id']=f['id'].replace('vaarweg_markering_drijvend.','')
        
            t,c=type_cat(p['obj_kleur_'],p['v_tt_c'])
            tags['seamark:type']=t
            if c: tags[f'seamark:{t}:category']=c
            tags['seamark:buoy_lateral:system']='iala-a' if 'lateral' in t else None
            tags[f'seamark:{t}:shape']=shape(p['obj_vorm_c'])
            tags[f'seamark:{t}:colour']=color(p['obj_kleur_'])
            tags[f'seamark:{t}:colour_pattern']=pattern(p['kleurpatr_'])
            tags['seamark:topmark:shape']=topmark(p['v_tt_c'])
            tags['seamark:topmark:colour']=color(p['tt_kleur_c'])
            tags['seamark:topmark:colour_pattern']=pattern(p['tt_pat_c'])
            tags['seamark:light:colour']=color(p['licht_kl_c'])
            tags['seamark:light:character']=light_chr(p['sign_kar_c'])
            tags['seamark:light:period']=light_per(p['sign_perio'])
            tags['seamark:light:group']=light_grp(p['sign_gr_c'])
    
            points.append(tags)
                    
            if 0:
                print(json.dumps(tags,indent=2))
                print(json.dumps(f,indent=2))
                break
        except:
            print(json.dumps(f,indent=2))
            raise
    
    
    #print(json.dumps(points[0], indent=2))
    #print(len(points),'points')

    return points

In [ ]:
def update(n, p):
    ll=[float(n.attr[a]) for a in ('lat','lon')]
    name=n.find("tag[k='seamark:name']").attr['v']
    log=[]
    #print(n)
    #print(json.dumps(p, indent=2))

    d=distance(ll,p['ll'])
    if d>5 or isnan(d):
        n.attr['lat'],n.attr['lon']=[str(x) for x in p['ll']]
        log.append(('MOV', p['ll'], distance(ll,p['ll'])))
        
    for k,v in p.items():
        if k.startswith('seamark') and 'source' not in k:
            #print(k,'=',v)
            tag=n.find(f"tag[k='{k}']")
            if tag:
                if not v:
                    log.append(('DEL', tag))
                    tag.remove()
                elif tag.attr["v"]!=v:
                    assert v
                    if k=='seamark:type':
                        #assert tag.attr['v']==v, (k,v,tag.attr['v'],name)
                        log.append(('TYPE CHANGED',name,k,v,tag.attr['v']))
                    tag.attr['v']=v
                    log.append(('MOD',tag))
            elif v:
                tag=pq(f'<tag k="{k}" v="{v}" />')
                n.append(tag)
                log.append(('ADD',tag))
                
    if log: 
        n.attr['action']='modify'
                    
        tag=n.find("tag[k='source']")
        if tag: tag.remove()
        tag=n.find("tag[k='seamark:source']")
        if tag: tag.remove()
        tag=n.find("tag[k='seamark:source:id']")
        if tag: tag.remove()
        s=p['seamark:source']
        id=p['seamark:source:id']
        pq(f'<tag k="seamark:source" v="{s}" />').append_to(n)
        pq(f'<tag k="seamark:source:id" v="{id}" />').append_to(n)

        
        print('node',n.attr['id'],ll,name)
        for l in log:
            print('    ',*[str(s).strip() for s in l])
            
        #print(n)

In [ ]:
def update_osm(infile, points, outfile):
    x=pq(filename=infile)
    bounds=x('osm bounds')
    bounds=[float(bounds.attr[a]) for a in ('minlat','maxlat','minlon','maxlon')]
    #print(bounds)
    #print(points[0])
    data=list(filter(lambda p: bounds[0]<p['ll'][0]<bounds[1] and bounds[2]<p['ll'][1]<bounds[3], points))
    #print(len(data),'points')
    
    nodes=x('node')
    #print(nodes)
    for e in nodes:
        n=pq(e)
        if n.find("tag[k='seamark:type']"):
            #print(n)
            ll=[float(n.attr[a]) for a in ('lat','lon')]
            if not (bounds[0]<ll[0]<bounds[1] and bounds[2]<ll[1]<bounds[3]): continue
            src=n.find("tag[k='seamark:source']").attr['v'] or ''
            id=n.find("tag[k='seamark:source:id']").attr['v']
            name=n.find("tag[k='seamark:name']").attr['v']
    
            #print(ll,name,src,id)
            p=[]
            if src and id:
                p=list(filter(lambda p: distance(ll,p['ll'])<=1000 and src.startswith(p['seamark:source'].split()[0]) and id==p['seamark:source:id'], data))
            if not p:
                p=list(filter(lambda p: distance(ll,p['ll'])<=500 and name==p['seamark:name'], data))
            if not p:
                p=list(filter(lambda p: distance(ll,p['ll'])<=100, data))
            #print(json.dumps(p, indent=2))
            
            assert len(p)<=1
            if len(p)==1:
                p=p[0]
                data.remove(p)
                if not n.attr['action']: 
                    update(n, p)
    
    print('-'*100)
    print('UNMATCHED',len(data))     
    
    for i,p in enumerate(data,1):
        n=pq(f'<node id="{-i}" visible="true" lat="nan" lon="nan"/>')
        update(n,p)
        x('osm').prepend(n)
        
    with open(outfile,'w') as f:
        f.write(str(x))

In [ ]:
data=load_data('vwm/drijvend.json')
update_osm('xxx.osm', data, 'yyy.osm')